In [15]:
import polars as pl
import pandas as pd
import numpy as np
import pybaseball as pb


In [16]:
p1 = 605400 #Aaron Nola
p2 = 668873 #Caleb Killian

p1_data = pb.statcast_pitcher("2024-09-23", "2024-09-23", player_id=p1)
p2_data = pb.statcast_pitcher("2024-09-23", "2024-09-23", player_id=p2)
pitch_data = (pl.from_pandas(pd.concat([p1_data, p2_data]))
                .with_columns(pl.concat_str([pl.col("game_pk"),
                                            pl.col("home_team"),
                                            pl.col("pitcher"),
                                            pl.col("at_bat_number"),
                                            pl.col("pitch_number")],
                              separator="-").alias("pitch_id")))


Gathering Player Data
Gathering Player Data


In [17]:
info_features = ["game_date", "game_pk", "home_team", "away_team", "player_name"]
event_features = ["events", "description", "type", "bb_type", "balls", "strikes", "inning", "outs_when_up", "des", "batter", "pitcher"]
release_pitcher_features = ["release_pos_x", "release_pos_y", "release_pos_z", "release_extension", "arm_angle"]
release_ball_features = ["release_speed", "release_spin", "spin_axis"]
traj_ball_features = ["pfx_x", "pfx_z", "vx0", "vy0", "vz0", "ax", "ay", "az"]
plate_ball_features = ["plate_x", "plate_z"]
pitch_id = ["pitch_id"]

In [18]:
release_frame = pitch_data.select(release_pitcher_features + ["pitcher", "pitch_type", "pitch_id"])

In [19]:
release_frame_agg

NameError: name 'release_frame_agg' is not defined

In [ ]:
vals = release_frame.filter(pl.col("pitch_type") == "KC").filter(pl.col("pitcher") == 605400).select("release_pos_z")


plot_histogram(vals, bins=10)

In [ ]:
vals1 = release_frame.filter(pl.col("pitch_type") == "KC").filter(pl.col("pitcher") == 605400).select("release_pos_y")
vals2 = release_frame.filter(pl.col("pitch_type") == "KC").filter(pl.col("pitcher") == 605400).select("release_pos_x")

plot_scatter(vals1, vals2)


In [71]:
import polars as pl

def distance_stats_from_centroid(df, x_col, y_col, z_col, group_cols):
    # Create expressions for the centroid calculations
    centroid_x = pl.col(x_col).mean().alias("centroid_x")
    centroid_y = pl.col(y_col).mean().alias("centroid_y")
    centroid_z = pl.col(z_col).mean().alias("centroid_z")
    
    # Expression for calculating Euclidean distance
    distance_expr = (
        ((pl.col(x_col) - pl.col("centroid_x")) ** 2 +
         (pl.col(y_col) - pl.col("centroid_y")) ** 2 +
         (pl.col(z_col) - pl.col("centroid_z")) ** 2)
        .sqrt()
        .alias("distance")
    )
    
    # Group, calculate centroids, distances, mean and standard deviation
    result = (
        df
        .group_by(group_cols)
        .agg(
            centroid_x,
            centroid_y,
            centroid_z
        )
        .join(df, on=group_cols)
        .with_columns([
            distance_expr
        ])
        .group_by(group_cols)
        .agg([
            pl.col("distance").mean().alias("mean_distance"),
            pl.col("distance").std().alias("std_dev_distance")
        ])
    )
    
    return result

# Example usage:
# data = pl.DataFrame({
#     'x': [1, 2, 3, 4, 5, 6],
#     'y': [7, 8, 9, 10, 11, 12],
#     'z': [13, 14, 15, 16, 17, 18],
#     'group1': ['A', 'A', 'A', 'B', 'B', 'B'],
#     'group2': [1, 1, 2, 2, 3, 3]
# })
# result = distance_stats_from_centroid(data, 'x', 'y', 'z', ['group1', 'group2'])

In [72]:
res = distance_stats_from_centroid(pitch_data, "release_pos_x", "release_pos_y", "release_pos_z", ["pitcher", "pitch_type"])

In [ ]:
p1_data = pb.statcast_pitcher("2024-09-23", "2024-09-23", player_id=p1)


In [1]:
import pybaseball as pb
import polars as pl


In [ ]:
seasons = [2021, 2022, 2023, 2024]
season_list = []

for season in seasons:
    print(season)
    season_frame = pl.from_pandas(pb.pitching_stats_bref(season))
    season_frame = (season_frame
                        .filter((pl.col("Lev") == "Maj-NL") | (pl.col("Lev") == "Maj-AL"))
                        .select(["mlbID", "G", "IP", "ERA", "WHIP", "SO9"])
                        .with_columns(pl.lit(season).alias("season"))
                        .with_columns(pl.col("mlbID").cast(pl.Int64)))
    season_list.append(season_frame)

frame = pl.concat(season_list)
frame = frame.rename({col: col.lower() for col in frame.columns})
players = frame["mlbid"].unique().to_list()
player_info = pb.playerid_reverse_lookup(player_ids=players)
player_info = player_info[["key_mlbam", "name_last", "name_first", "mlb_played_first", "mlb_played_last"]]
player_info = pl.from_pandas(player_info)
player_info = (player_info.with_columns((pl.col("name_first") + " " + pl.col("name_last")).alias("player_name"))
                            .with_columns(pl.col("player_name").str.to_titlecase().alias("player_name")))
player_info = player_info.rename({"key_mlbam": "mlbid"}).select(["mlbid", "player_name", "mlb_played_first", "mlb_played_last"])
frame = frame.join(player_info, on = "mlbid", how = "left")
frame = frame.select(["mlbid", "player_name", "season", "g", "ip", "era", "whip", "so9", "mlb_played_first", "mlb_played_last"])
frame.write_parquet("pitchers.parquet")

In [1]:
import polars as pl
import pybaseball as pb

def get_season_start_end(season: int = 2024) -> list:
    """Returns a list with the first and last day of the regular season. Includes Tokyo + Korea series.""" 

    if season < 2021:
        print("Season must be >= 2021")
        return
    
    if season == 2021:
        return ["2021-04-01", "2021-10-03"]

    if season == 2022:
        return ["2022-04-07", "2022-10-05"]

    if season == 2023:
            return ["2023-03-30", "2023-10-01"]

    if season == 2024:
            return ["2024-03-20", "2024-10-03"]
    
    if season == 2025:
            return ["2025-03-18", "2025-09-28"]
    

In [6]:
pitchers = pl.read_parquet("../src/ninjalytics/data/pitchers.parquet")

In [20]:
from tqdm import tqdm

seasons = [2021, 2022, 2023, 2024]
season_list = []

for season in seasons:
    print(seasons)
    dates = get_season_start_end(season = season)
    start = dates[0]
    stop = dates[1]
    season_pitchers = (pitchers
                        .filter(pl.col("season") == season)
                        .filter(pl.col("ip") >= 20)
                        .select(["mlbid"])
                        .to_series()
                        .to_list())
    season_pitcher_list = []
    
    for pitcher in tqdm(season_pitchers):
        p = pb.statcast_pitcher(start, stop, player_id=pitcher)
        p = pl.from_pandas(p)
        p = p.with_columns(pl.lit(season).alias("season"))
        season_pitcher_list.append(p)

    season_pitcher_frame = pl.concat(season_pitcher_list)
    season_pitcher_frame.write_parquet(f"../src/ninjalytics/data/pitches_{season}.parquet")


    




[2021, 2022, 2023, 2024]


  0%|          | 1/520 [00:00<01:18,  6.63it/s]

Gathering Player Data
Gathering Player Data


  1%|          | 3/520 [00:00<01:24,  6.12it/s]

Gathering Player Data
Gathering Player Data


...

Gathering Player Data


 17%|█▋        | 88/520 [06:15<28:21,  3.94s/it]

Gathering Player Data


 17%|█▋        | 88/520 [06:18<30:58,  4.30s/it]


ParserError: Error tokenizing data. C error: Expected 1 fields in line 13, saw 2


In [9]:

season_pitchers = (pitchers
                    .filter(pl.col("season") == 2022)
                    .filter(pl.col("ip") >= 20)
                    .select(["mlbid"])
                    .to_series()
                    .to_list())

In [12]:
season_pitchers.to_series().to_list()

[676265,
 656061,
 650556,
 642758,
 592094,
 672851,
 669211,
 645261,
 669920,
 641302,
 663465,
 622075,
 621237,
 502624,
 666120,
 542881,
 644364,
 502042,
 685503,
 676879,
 668933,
 665871,
 641329,
 621383,
 453268,
 669618,
 605130,
 606930,
 598264,
 642545,
 605135,
 657508,
 642585,
 669358,
 670280,
 595881,
 656222,
 571479,
 678394,
 621244,
 641360,
 669456,
 656234,
 621389,
 621112,
 542947,
 621366,
 502202,
 680694,
 666374,
 518489,
 623211,
 605154,
 689225,
 656257,
 641401,
 686613,
 664141,
 677865,
 663460,
 621111,
 518516,
 607481,
 605164,
 656271,
 669203,
 665795,
 650893,
 471911,
 641447,
 528748,
 650895,
 666721,
 612434,
 656302,
 570666,
 468504,
 605177,
 547973,
 608638,
 643256,
 518553,
 542585,
 650644,
 664199,
 661403,
 605182,
 502171,
 543037,
 669395,
 517008,
 672710,
 571578,
 641482,
 676710,
 622503,
 640444,
 621016,
 456501,
 641501,
 506433,
 605200,
 656354,
 608328,
 660853,
 660787,
 594798,
 672282,
 446321,
 664747,
 621242,
 